In [1]:
#Importing libraries
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,date_format
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Setting pandas data frame options to have a precision of 2
pd.set_option('display.precision', 2)


In [4]:
#Setting up and initializing spark session
def init_spark():
  spark = SparkSession.builder\
    .appName("tevin_data")\
    .config("spark.jars", "../../spark/jars/ojdbc8.jar")\
    .master("spark://spark-master:7077")\
    .config("spark.executor.memory", "2g")\
    .config("spark.cores.max", 1)\
    .getOrCreate()
    
  sc = spark.sparkContext
  return spark,sc

In [5]:
spark,sc = init_spark()

In [7]:
#Connecting to the db
url = 'jdbc:oracle:thin:@copkdnas-c1-scan:1528/COPKBI'
properties = {
    "user": "toloo",
    "password": "Prospects08$",
    "driver": "oracle.jdbc.driver.OracleDriver"}


In [8]:
## SQL Query

sql="""
(
select ACCOUNTID ,SUM(BOOKEDBALANCE) BOOKEDBALANCE, SUM(CLEAREDBALANCE) CLEAREDBALANCE 
from coop_stg.bi_FINANCE_STATUS_LIST_FILE GROUP BY ACCOUNTID
) script 
"""

In [9]:
## Running spark job 
df= spark.read.jdbc(url=url, table=sql, properties=properties)

In [10]:
# Checking the column properties
df.printSchema()

root
 |-- ACCOUNTID: string (nullable = true)
 |-- BOOKEDBALANCE: decimal(38,10) (nullable = true)
 |-- CLEAREDBALANCE: decimal(38,10) (nullable = true)



In [11]:
#storing output as a dataframe
pdf = df.toPandas()
pdf.head()

KeyboardInterrupt: 

In [ ]:
#Commence here
#Storing the output to a db table
spdf = spark.createDataFrame(pdf) 
spdf.write.jdbc(url=url, table="db_schema.table_name", mode='append', properties=properties)# specify the target table name